### Overview of Google Cloud OAuth 2.0
1. Our web App generates authorization url
2. User clicks authorization url and grants permission to our web app
3. User is redirected by Google to our web app. 
4. The redirected url has authorization code in it. So our web app gets the code and trade it with Google for tokens 
5. Our web app uses tokens to work.

### Docs
* https://developers.google.com/identity/protocols/oauth2/web-server
* https://developers.google.com/youtube/v3/guides/auth/server-side-web-apps
* https://google-auth-oauthlib.readthedocs.io/en/latest/reference/google_auth_oauthlib.flow.html

In [ ]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>")) 

In [ ]:
# -*- coding: utf-8 -*-

import os, json, flask, requests

import google.oauth2.credentials
import google_auth_oauthlib.flow
import googleapiclient.discovery
 
CLIENT_SECRETS_FILE = "oauth2_client_secret.json"
SCOPES = ['https://www.googleapis.com/auth/youtube.upload', 
          'https://www.googleapis.com/auth/youtube.force-ssl']
API_SERVICE_NAME = 'youtube'
API_VERSION = 'v3'

app = flask.Flask(__name__)
app.secret_key = "abcdefg1234567"


@app.route('/')
def index():
    if 'credentials' in flask.session:
        with open('oauth2_user_credentials.json', 'w') as fw:
            json.dump(flask.session['credentials'], fp=fw)
        return flask.jsonify({'Note':'User Crendentials saved'})
    else:
        return flask.redirect('authorize')


@app.route('/authorize')
def authorize(): 
    flow = google_auth_oauthlib.flow.Flow.from_client_secrets_file(CLIENT_SECRETS_FILE, scopes=SCOPES) 
    flow.redirect_uri = flask.url_for('oauth2callback', _external=True) 
    # 1. Generate authorization_url for the user to give permission
    authorization_url, flask.session['state'] = flow.authorization_url(
        access_type='offline', 
        include_granted_scopes='true', 
        prompt='consent'
    ) 
    # 2. User is redirected to Google Login to give permission. 
    return flask.redirect(authorization_url)


@app.route('/oauth2callback')
def oauth2callback():
    # 3. Google redirects user to this endpoint, in which user hands over authorization code to the app
    authorization_response = flask.request.url 
    flow = google_auth_oauthlib.flow.Flow.from_client_secrets_file(
        CLIENT_SECRETS_FILE, 
        scopes=SCOPES
    )
    # if you do not specify redirect_uri, you get the error of Missing parameter: redirect_uri 
    flow.redirect_uri = flask.url_for('oauth2callback', _external=True) 
    # 4. App trades authorization code with Goolge for refresh token and access token 
    flow.fetch_token(authorization_response=authorization_response) 
    flask.session['credentials'] = credentials_to_dict(flow.credentials) 

    return flask.redirect(flask.url_for('index'))


@app.route('/revoke')
def revoke():
    if 'credentials' not in flask.session:
        return ('You need to <a href="/authorize">authorize</a> before ' +
            'testing the code to revoke credentials.')

    credentials = google.oauth2.credentials.Credentials(**flask.session['credentials'])

    revoke = requests.post('https://oauth2.googleapis.com/revoke',
        params={'token': credentials.token},
        headers = {'content-type': 'application/x-www-form-urlencoded'})

    status_code = getattr(revoke, 'status_code')
    if status_code == 200:
        return('Credentials successfully revoked.')
    else:
        return('An error occurred.')


@app.route('/clear')
def clear_credentials():
    if 'credentials' in flask.session:
        del flask.session['credentials']
    return ('Credentials have been cleared.<br><br>')

def credentials_to_dict(credentials):
    return {'token': credentials.token,
          'refresh_token': credentials.refresh_token,
          'token_uri': credentials.token_uri,
          'client_id': credentials.client_id,
          'client_secret': credentials.client_secret,
          'scopes': credentials.scopes}


if __name__ == '__main__':  
    server_cert_letsencrypt  = 'fullchain.pem'  # secret of my apache and flask site
    server_key_letsencrypt   = 'privkey.pem'    # secret of my apache and flask site
    app.run('0.0.0.0','8765', ssl_context=(server_cert_letsencrypt, server_key_letsencrypt)) 